In [1]:
# 파이토치 호출
import torch
import torch.nn as nn

# 토치비전 transform 및 데이터셋 가져오기
from torchvision.transforms import transforms
from torchvision.datasets import FashionMNIST

# 데이터 로더 가져오기
from torch.utils.data.dataloader import DataLoader

# 최적화 알고리즘 가져오기 
from torch.optim import optimizer

In [2]:
# 데이터 transform 적용
data_transforms = transforms.Compose([
    transforms.Resize((227,227)),
    transforms.ToTensor()
])

In [3]:
# 데이터 불러오기(fashion MNIST 사용)

path = '/home/helpme/data/FashionMNIST'

train_data = FashionMNIST(
    root = path,
    train=True,
    download=True,
    transform=data_transforms
)

test_data = FashionMNIST(
    root = path,
    train=False,
    download=True,
    transform=data_transforms
)

In [4]:
batch_size = 128
num_workers = 4

train_dataloader = DataLoader(
    train_data, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=num_workers)

test_dataloader = DataLoader(
    test_data, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=num_workers)

In [5]:
(train_dataloader.dataset)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: /home/helpme/data/FashionMNIST
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(227, 227), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [6]:
print(f'train data 개수 : {len(train_dataloader.dataset)}')
print(f'test data 개수 : {len(test_dataloader.dataset)}')

for x, y in test_dataloader:
    print(f'x shape (N, C, H, W): {x.shape}')
    print(f'y shape : {y.shape, y.dtype}')
    break

train data 개수 : 60000
test data 개수 : 10000
x shape (N, C, H, W): torch.Size([128, 1, 227, 227])
y shape : (torch.Size([128]), torch.int64)


In [7]:
# 모델 만들기
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2)
            
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        
        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(in_features=256*6*6, out_features=4096),
            nn.ReLU(),
            nn.Dropout(0.5)            
        )
        self.fc2 = nn.Sequential(
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Dropout(0.5)        
        )

        self.fc3 = nn.Sequential(
            nn.Linear(in_features=4096, out_features=10)
        )
        

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)

        x = x.view(x.size(0), -1) 

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x


In [8]:
model = AlexNet()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')
model.to(device)


Using cuda device


AlexNet(
  (layer1): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (layer5): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Li

In [9]:
from torchsummary import summary

summary(model, (1,227,227), batch_size=16)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [16, 96, 55, 55]          11,712
              ReLU-2           [16, 96, 55, 55]               0
 LocalResponseNorm-3           [16, 96, 55, 55]               0
         MaxPool2d-4           [16, 96, 27, 27]               0
            Conv2d-5          [16, 256, 27, 27]         614,656
              ReLU-6          [16, 256, 27, 27]               0
 LocalResponseNorm-7          [16, 256, 27, 27]               0
         MaxPool2d-8          [16, 256, 13, 13]               0
            Conv2d-9          [16, 384, 13, 13]         885,120
             ReLU-10          [16, 384, 13, 13]               0
           Conv2d-11          [16, 384, 13, 13]       1,327,488
             ReLU-12          [16, 384, 13, 13]               0
           Conv2d-13          [16, 256, 13, 13]         884,992
             ReLU-14          [16, 256,

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [11]:
criterion.to(device)

CrossEntropyLoss()

In [12]:
total_step = len(train_dataloader)

In [13]:
num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        	
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        		
        if (i+1) % 200 == 0:
            # print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        	# 	           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            print("Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, i * len(images), len(train_dataloader.dataset),
                100. * i / len(train_dataloader), loss.item()))


Train Epoch:0 [25472/60000 (42%)]	Loss: 0.468370
Train Epoch:0 [51072/60000 (85%)]	Loss: 0.367541
Train Epoch:1 [25472/60000 (42%)]	Loss: 0.222629
Train Epoch:1 [51072/60000 (85%)]	Loss: 0.296512
Train Epoch:2 [25472/60000 (42%)]	Loss: 0.279272
Train Epoch:2 [51072/60000 (85%)]	Loss: 0.339498
Train Epoch:3 [25472/60000 (42%)]	Loss: 0.438222
Train Epoch:3 [51072/60000 (85%)]	Loss: 0.192673
Train Epoch:4 [25472/60000 (42%)]	Loss: 0.276463
Train Epoch:4 [51072/60000 (85%)]	Loss: 0.341105
Train Epoch:5 [25472/60000 (42%)]	Loss: 0.188108
Train Epoch:5 [51072/60000 (85%)]	Loss: 0.304692
Train Epoch:6 [25472/60000 (42%)]	Loss: 0.119967
Train Epoch:6 [51072/60000 (85%)]	Loss: 0.140727
Train Epoch:7 [25472/60000 (42%)]	Loss: 0.175114
Train Epoch:7 [51072/60000 (85%)]	Loss: 0.161838
Train Epoch:8 [25472/60000 (42%)]	Loss: 0.115868
Train Epoch:8 [51072/60000 (85%)]	Loss: 0.194201
Train Epoch:9 [25472/60000 (42%)]	Loss: 0.194191
Train Epoch:9 [51072/60000 (85%)]	Loss: 0.125336


In [14]:
# testing
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 90.98 %
